In [ ]:
import numpy as np
import random
import pandas as pd

import sys
sys.path.append('C:\\Users\\PC\\Documents\\GitHub\\Ugur-dynamic-scheduling\\functions')
import vizings_schedule
from yeni_model import yeni_model
from weighted_days_model import weighted_days_model
from generate_results import generate_results, generate_results2, generate_results3


resu = [
    [0, 0, 0, 0, 0, 1],
    [3, 0, 0, 1, 0, 0],
    [3, 3, 0, 0, 1, 0],
    [3, 1, 3, 0, 0, 0],
    [3, 3, 1, 3, 0, 0],
    [1, 3, 3, 3, 0, 0]
]
resu = np.array(resu)
results_df2 = pd.DataFrame(resu)

In [ ]:
big_list_v_min = []
big_list_v_max = []
big_list_c_min = []
big_list_c_max = []
big_list_wm = []
big_list_m = []
for n in range(4, 20, 2):
    teams = list(range(1,n+1))
    days = list(range(1,n))
    #results_df = pd.read_excel('C:\\Users\\PC\\Desktop\\match_resultlar.xlsx', sheet_name='Germany second half')
    results_df = generate_results2(teams)
    result_array = np.zeros((n, n), dtype=int)
    for _, row in results_df.iterrows():
        result = row['Result']
        team1 = row['Team 1']
        team2 = row['Team 2']

        index_team1 = teams.index(team1)
        index_team2 = teams.index(team2)

        if team1 == result:
            result_array[index_team1, index_team2] = 3
        elif team2 == result:
            result_array[index_team2, index_team1] = 3
        else:
            result_array[index_team1, index_team2] = 1
            result_array[index_team2, index_team1] = 1
    result_matrix = result_array
    #days = list(range(n-1,0,-1))
    #weights_list = [round(1/i) for i in range(1, n)]
    #print(weights_list)
    number_of_matches = int((n*(n-1))/2)
    print(result_matrix)
    simulation = []
    canon_simulation = []

    for i in range(1,10001):
        timetable = vizings_schedule.vizing(n)
        each_sim = vizings_schedule.calculate_obj_value(n, timetable, result_matrix)
        simulation.append(each_sim)
    big_list_v_min.append(min(simulation))
    big_list_v_max.append(max(simulation))

    for j in range(1,10001):
        canon = vizings_schedule.canonical(n)
        canon_shuffle = vizings_schedule.rndAllRounds(canon, n-1)
        obj_canon= vizings_schedule.calculate_obj_value(n, canon_shuffle, result_matrix)
        canon_simulation.append(obj_canon)
    big_list_c_min.append(min(canon_simulation))
    big_list_c_max.append(max(canon_simulation))


    # Burada result matrixi oluşturuyorum.
    result_array = np.zeros((n, n), dtype=int)
    for _, row in results_df.iterrows():
        result = row['Result']
        team1 = row['Team 1']
        team2 = row['Team 2']

        index_team1 = teams.index(team1)
        index_team2 = teams.index(team2)

        if team1 == result:
            result_array[index_team1, index_team2] = 3
        elif team2 == result:
            result_array[index_team2, index_team1] = 3
        else:
            result_array[index_team1, index_team2] = 1
            result_array[index_team2, index_team1] = 1
    print(result_array)
    data = pd.DataFrame(result_array, columns=teams, index=teams)

    output, d1_df, d2_df, y_df, z_df, obj_value2, l, l2= weighted_days_model(teams, days, data)
    sonuc = np.array(output)
    ratings = {i: 0 for i in teams}

    objective_values = []


    for d in days:
        obj_count =0
        for row in sonuc:
            day,team1,team2 = row
            if d==day:
                obj_count +=(abs(ratings[team1] - ratings[team2]))
                ratings[team1] += result_matrix[team1-1,team2-1]
                ratings[team2] += result_matrix[team2-1,team1-1]
        print(ratings)
        objective_values.append(obj_count)
    a = sum(objective_values)
    big_list_wm.append(a)


    output2, d1_df, d2_df, y_df, z_df, obj_value2, l, l2= yeni_model(teams, days, data)
    sonuc2 = np.array(output2)
    ratings = {i: 0 for i in teams}

    objective_values = []


    for d in days:
        obj_count =0
        for row in sonuc2:
            day,team1,team2 = row
            if d==day:
                obj_count +=(abs(ratings[team1] - ratings[team2]))
                ratings[team1] += result_matrix[team1-1,team2-1]
                ratings[team2] += result_matrix[team2-1,team1-1]
        print(ratings)
        objective_values.append(obj_count)
    b = sum(objective_values)
    big_list_m.append(b)



In [ ]:
print(big_list_m)
print(big_list_wm)
print(big_list_v_min)
print(big_list_c_min)
#print(big_list_v_max)
#print(big_list_c_max)
#print(simulation)
#print(canon_simulation)

big_list_v = []
big_list_c = []
big_list_m = []

for n in range(10,12,2):
    teams = list(range(1,n+1))
    days = list(range(1,n))
    #days = list(range(n-1,0,-1))
    #weights_list = [round(1/i) for i in range(1, n)]
    #print(weights_list)
    number_of_matches = int((n*(n-1))/2)
    result_df, result_matrix = vizings_schedule.generate_results(teams)
    print(result_matrix)
    simulation = []
    canon_simulation = []

    for i in range(1,100001):
        timetable = vizings_schedule.vizing(n)
        each_sim = vizings_schedule.calculate_obj_value(n, timetable, result_matrix)
        simulation.append(each_sim)
    big_list_v.append(simulation)

    for j in range(1,100001):
        canon = vizings_schedule.canonical(n)
        canon_shuffle = vizings_schedule.rndAllRounds(canon, n-1)
        obj_canon= vizings_schedule.calculate_obj_value(n, canon_shuffle, result_matrix)
        canon_simulation.append(obj_canon)
    big_list_c.append(canon_simulation)


    # Burada result matrixi oluşturuyorum.
    result_array = np.zeros((n, n), dtype=int)
    for _, row in result_df.iterrows():
        result = row['Result']
        team1 = row['Team 1']
        team2 = row['Team 2']

        index_team1 = teams.index(team1)
        index_team2 = teams.index(team2)

        if team1 == result:
            result_array[index_team1, index_team2] = 3
        elif team2 == result:
            result_array[index_team2, index_team1] = 3
        else:
            result_array[index_team1, index_team2] = 1
            result_array[index_team2, index_team1] = 1
    print(result_array)
    data = pd.DataFrame(result_array, columns=teams, index=teams)

    output2, d1_df, d2_df, y_df, z_df, obj_value2, l, l2= weighted_days_model(teams, days, data)
    sonuc = np.array(output2)
    ratings = {i: 0 for i in teams}

    objective_values = []


    for d in days:
        obj_count =0
        for row in sonuc:
            day,team1,team2 = row
            if d==day:
                obj_count +=(abs(ratings[team1] - ratings[team2]))
                ratings[team1] += result_matrix[team1-1,team2-1]
                ratings[team2] += result_matrix[team2-1,team1-1]
        print(ratings)
        objective_values.append(obj_count)
    a = sum(objective_values)
    big_list_m.append(a)

In [ ]:
print(big_list_v)
print(big_list_c)

v=np.array(big_list_v)
c=np.array(big_list_c)
print(np.min(v, axis=1), np.mean(v, axis=1), np.max(v, axis=1))
print(np.min(c, axis=1), np.mean(c, axis=1), np.max(c, axis=1))

print(big_list_m)

In [ ]:
# Burada result matrixi oluşturuyorum.
result_array = np.zeros((n, n), dtype=int)
for _, row in result_df.iterrows():
    result = row['Result']
    team1 = row['Team 1']
    team2 = row['Team 2']

    index_team1 = teams.index(team1)
    index_team2 = teams.index(team2)

    if team1 == result:
        result_array[index_team1, index_team2] = 3
    elif team2 == result:
        result_array[index_team2, index_team1] = 3
    else:
        result_array[index_team1, index_team2] = 1
        result_array[index_team2, index_team1] = 1
print(result_array)
data = pd.DataFrame(result_array, columns=teams, index=teams)


output, d1_df, d2_df, y_df, z_df, obj_value, l, l2= yeni_model(teams, days, data)
ratings1 = {i: 0 for i in teams}
sonuc1 = np.array(output)



for d in days:
    obj_count =0
    for row in sonuc1:
        day,team1,team2 = row
        if d==day:
            obj_count +=(abs(ratings1[team1] - ratings1[team2]))
            ratings1[team1] += result_matrix[team1-1,team2-1]
            ratings1[team2] += result_matrix[team2-1,team1-1]

In [ ]:
# Burada result matrixi oluşturuyorum.
result_array = np.zeros((n, n), dtype=int)
for _, row in result_df.iterrows():
    result = row['Result']
    team1 = row['Team 1']
    team2 = row['Team 2']

    index_team1 = teams.index(team1)
    index_team2 = teams.index(team2)

    if team1 == result:
        result_array[index_team1, index_team2] = 3
    elif team2 == result:
        result_array[index_team2, index_team1] = 3
    else:
        result_array[index_team1, index_team2] = 1
        result_array[index_team2, index_team1] = 1
print(result_array)
data = pd.DataFrame(result_array, columns=teams, index=teams)

output2, d1_df, d2_df, y_df, z_df, obj_value2, l, l2= weighted_days_model(teams, days, data)
sonuc = np.array(output2)
ratings = {i: 0 for i in teams}

objective_values = []


for d in days:
    obj_count =0
    for row in sonuc:
        day,team1,team2 = row
        if d==day:
            obj_count +=(abs(ratings[team1] - ratings[team2]))
            ratings[team1] += result_matrix[team1-1,team2-1]
            ratings[team2] += result_matrix[team2-1,team1-1]
    print(ratings)
    objective_values.append(obj_count)
a = sum(objective_values)

In [ ]:
print(simulation)
simulation = np.array(simulation)
mean = np.mean(simulation)
print(mean)
print(obj_canon)
print(a)
#print(obj_value)
print(np.min(simulation))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data
x = list(range(1, len(simulation)+1))

# Calculate minimum, average, and maximum
min_values = np.min(simulation)
avg_values = np.mean(simulation)
max_values = np.max(simulation)

# Create a figure and axis
fig, ax = plt.subplots()

# Plot each line without markers
ax.plot(x, [min_values]*len(x), label='Minimum')
ax.plot(x, [avg_values]*len(x), label='Average')
ax.plot(x, [max_values]*len(x), label='Maximum')
ax.plot(x, [obj_canon]*len(x), label='canonical')
ax.plot(x, [a]*len(x), label='weighted model')
#ax.plot(x, [obj_value]*len(x), label='model')

# Set labels and title
ax.set_xlabel('Data Point')
ax.set_ylabel('Value')
ax.set_title('Line Plot')

# Add legend
ax.legend()

# Show the plot
plt.show()


teams = list(range(1,n+1))
days = list(range(1,n))
non_zero_entries = timetable[1:, 1:]

new_format_list = []

# Iterate through the timetable matrix to extract day, team1, and team2 information
for team1 in range(n):
    for team2 in range(n):
        day = non_zero_entries[team2, team1]
        new_format_list.append([day, team1, team2])

# Convert the list to a NumPy array
new_format_array = np.array(new_format_list)
new_format_array[:, 1:] += 1
new_format_array = new_format_array[new_format_array[:, 0].argsort()]
new_format_array = new_format_array[new_format_array[:, 2] < new_format_array[:, 1]]
solution_df = pd.DataFrame(new_format_array, columns=['Day', 'Team 1', 'Team 2'])
relevant_matches = pd.DataFrame(columns=['Team 1', 'Team 2', 'Day'])
for _, row in solution_df.iterrows():
    day = row['Day']
    team1 = row['Team 1']
    team2 = row['Team 2']
    if team1 < team2:
        relevant_matches = pd.concat([relevant_matches, pd.DataFrame({'Team 1': [team1], 'Team 2': [team2], 'Day': [day]})],
                                                    ignore_index=True)
    else:
        relevant_matches = pd.concat([relevant_matches, pd.DataFrame({'Team 1': [team2], 'Team 2': [team1], 'Day': [day]})],
                                                    ignore_index=True)
result_array = np.zeros((n, n), dtype=int)
for _, row in result_df.iterrows():
    result = row['Result']
    team1 = row['Team 1']
    team2 = row['Team 2']

    index_team1 = teams.index(team1)
    index_team2 = teams.index(team2)

    if team1 == result:
        result_array[index_team1, index_team2] = 3
    elif team2 == result:
        result_array[index_team2, index_team1] = 3
    else:
        result_array[index_team1, index_team2] = 1
        result_array[index_team2, index_team1] = 1
print(result_array)
data = pd.DataFrame(result_array, columns=teams, index=teams)



output, d1_df, d2_df, y_df, z_df, obj_value, l, l2= yeni_model(teams,days,data)

import matplotlib.pyplot as plt
import numpy as np

data_model = {4: 6, 6: 8, 8: 27, 10: 35, 12: 64, 14: 102, 20: 537}
data_sim = {4: 7.8, 6: 23.2, 8: 62.5, 10: 106.2, 12: 183, 14: 283.3, 20: 749.8}

# Extract keys and values
keys = list(data_model.keys())
values_model = list(data_model.values())
values_sim = list(data_sim.values())

# Create a bar chart
bar_width = 0.35
index = np.arange(len(keys))

fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figure size

bar1 = ax.bar(index, values_model, bar_width, label='Model', color='skyblue', edgecolor='black')
bar2 = ax.bar(index + bar_width, values_sim, bar_width, label='Mean of 1000 schedules', color='lightcoral', edgecolor='black')

# Set labels and title
ax.set_xlabel('Number of Teams', fontsize=14)
ax.set_ylabel('Objective Values', fontsize=14)
ax.set_title('Comparison Model with 1000 Random Schedules', fontsize=16)
ax.set_xticks(index + bar_width/2)
ax.set_xticklabels(keys)
ax.legend()

# Add grid lines
ax.grid(axis='y', linestyle='--', alpha=0.7)

# Add value annotations on top of the bars
for i, v in enumerate(values_model):
    ax.text(i, v + 10, str(v), ha='center', va='bottom', color='black', fontweight='bold', fontsize=10)

for i, v in enumerate(values_sim):
    ax.text(i + bar_width, v + 10, str(v), ha='center', va='bottom', color='black', fontweight='bold', fontsize=10)

# Show the plot
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()


obj_data = (7730, 7418, 630, 576, 558, 554, 536 ,528, 518, 508, 506, 504, 497, 484, 465, 460, 449, 442, 423, 414, 412, 408, 407, 406, 401, 400, 398, 397, 395, 393, 391, 389, 384, 382, 375, 369, 367, 361)
time_change = (12.1, 12.5, 15, 16, 87, 89, 148, 830, 861, 892, 893, 894, 932, 975, 1005, 2017, 2887, 2928, 2929, 2977, 3017, 3018, 3168, 3220, 3382, 3647, 3648, 3952, 4252, 6514, 6515, 6549, 6563, 6564, 7025, 7080,9485,9869)
print(len(obj_data))
print(len(time_change))
obj_data_dict = {i: obj_data[i] for i in range(len(obj_data))}
time_change_dict = {i: time_change[i] for i in range(len(time_change))}

# Create a numpy array with two columns
data_array = np.column_stack((time_change, obj_data))

print(data_array)



import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D

# Your data
obj_data = (7730, 7418, 630, 576, 558, 554, 536, 528, 518, 508, 506, 504, 497, 484, 465, 460, 449, 442, 423, 414, 412, 408, 407, 406, 401, 400, 398, 397, 395, 393, 391, 389, 384, 382, 375, 369, 367, 361, 361)
time_change = (12.1, 12.5, 15, 16, 87, 89, 148, 830, 861, 892, 893, 894, 932, 975, 1005, 2017, 2887, 2928, 2929, 2977, 3017, 3018, 3168, 3220, 3382, 3647, 3648, 3952, 4252, 6514, 6515, 6549, 6563, 6564, 7025, 7080, 9485, 9869, 14400)

# Create a numpy array with two columns
data_array = np.column_stack((time_change, obj_data))

# Set Seaborn style
sns.set(style="whitegrid")

# Create a figure and axis object
fig, ax = plt.subplots(figsize=(10, 6))

# Plot the data using Seaborn with specified colors
line = sns.lineplot(x=time_change, y=obj_data, color='skyblue', ax=ax)
scatter = sns.scatterplot(x=time_change, y=obj_data, color='red', ax=ax, s=40, zorder=10)

# Set the y-axis limits
ax.set_ylim(250, 800)

# Set plot labels and title
ax.set_xlabel('Time (seconds)')
ax.set_ylabel('Objective Value')
ax.set_title('Change of Objective Values throughout the iterations (18 teams)')

# Add legend with a note
legend_labels = {'first': 'First iteration: 7730', 'second': 'Second iteration: 7418'}

# Create custom legend handler for red dots
legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label=label) for label in legend_labels.values()]

# Add legend with custom elements
ax.legend(handles=[line, *legend_elements])

# Show plot
plt.show()


non_zero_entries = timetable[1:, 1:]

new_format_list = []

# Iterate through the timetable matrix to extract day, team1, and team2 information
for team1 in range(n):
    for team2 in range(n):
        day = non_zero_entries[team2, team1]
        new_format_list.append([day, team1, team2])

# Convert the list to a NumPy array
new_format_array = np.array(new_format_list)
new_format_array[:, 1:] += 1
new_format_array = new_format_array[new_format_array[:, 0].argsort()]
new_format_array = new_format_array[new_format_array[:, 2] < new_format_array[:, 1]]
solution_df = df = pd.DataFrame(new_format_array, columns=['Day', 'Team 1', 'Team 2'])

path = ('C:/Users/ug033207/vizings deneme.xlsx')
with pd.ExcelWriter(path, engine='xlsxwriter') as excel_writer:

# Write each DataFrame to a different sheet
    solution_df.to_excel(excel_writer, sheet_name='solution', index=False)
    result_df.to_excel(excel_writer, sheet_name='results', index=False)




new_format_array = np.hstack((new_format_array, np.zeros((new_format_array.shape[0], 1))))

# Iterate through each row in new_format_array
for row_idx in range(new_format_array.shape[0]):
    # Extract team1 and team2 from the current row
    team1 = int(new_format_array[row_idx, 1])
    team2 = int(new_format_array[row_idx, 2])

    # Find the corresponding row in results_df based on team1 and team2
    match_result = results_df[((results_df['Team 1'] == team1) & (results_df['Team 2'] == team2)) |
                               ((results_df['Team 1'] == team2) & (results_df['Team 2'] == team1))]['Result'].values

    # If a match result is found, update the new column in new_format_array
    if len(match_result) > 0:
        new_format_array[row_idx, -1] = match_result[0]